In [1]:
import pickle
import cv2
import os
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score    
from skimage.feature import local_binary_pattern

#global constants
ROI_SIZE_XY=200
MAX_IMAGES = 20
NAMES = ["Harry","Phil"]#,"Maureen"]

def getFiles(NAME):
    cascade_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    DATABASE_PATH = "./../Datastore/Face/"+NAME+"/hasFaces"
    images=[]
    labels=[]
    global MAX_IMAGES 
    local_MAX_IMAGES=MAX_IMAGES
    # Loop through all files in the directory
    for filename in os.listdir(DATABASE_PATH):
        local_MAX_IMAGES-=1      
        if filename.endswith('.jpg'):
            # Construct the file path
            filepath = os.path.join(DATABASE_PATH, filename)
            # Read the image
            img = cv2.imread(filepath)
            
            ##Filtering for better recognition
            greyscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = cascade_classifier.detectMultiScale(greyscale, 1.3, 5)            
            for (x,y,w,h) in faces:
                roi = img[y:y+ROI_SIZE_XY, x:x+ROI_SIZE_XY]                        
            images.append(roi)
            labels.append(NAMES.index(NAME))
    
        if (local_MAX_IMAGES<1):
            break
    return (images,labels)

def train_svm(features, labels):
    # Split data into training and testing subsets
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0)

    # Train SVM classifier
    svm_classifier = svm.SVC()
    svm_classifier.fit(X_train, y_train)
    return svm_classifier, X_test, y_test

##implement local binary pattern feature extraction
def extract_LBP_features(images):
    lbp_features = []

    for image in images:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        radius = 3
        n_points = 8 * radius
        lbp = local_binary_pattern(gray_image, n_points, radius, method='uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(2**n_points + 1), density=True)
        lbp_features.append(hist)

    lbp_features = np.array(lbp_features)
    return lbp_features

def main():
    ######START PROGRAM\\
    numerical_labels=[]
    for name in NAMES:
        try:
            index = NAMES.index(name)
            numerical_labels.append(index)
        except ValueError:
            print("Element not found")

    ###Gather images and labels in arrays
    images = []
    labels = []
    for name in NAMES:
        img, lab = getFiles(name)
        images+=(img)
        labels+=(lab)
        print(len(img),"Images added for",name)
    print(len(images),"Images added in total ")



    # Step 5: Testing/Evaluation
    def test_svm(svm_classifier, X_test, y_test):
        
        predictions = svm_classifier.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        print("Accuracy:", accuracy)
        
        
    # Extract LBP features
    print("DONE\n\nExtracting LBM features")
    LBP_features = extract_LBP_features(images)

    print("DONE\n\nTraining SVM classifier")
    # Train SVM classifier
    svm_classifier, X_test, y_test= train_svm(LBP_features, labels)

    print("DONE\n\nTesting SVM Classifier")
    # Test SVM classifier
    print(y_test)
    test_svm(svm_classifier, X_test, y_test)


    print("DONE\n\nSaving Classifier")
    # Create the label mapping dictionary
    label_mapping = {numerical_labels: label for numerical_labels, label in zip(numerical_labels, NAMES)}
    # Save the SVM classifier
    with open('svm_classifier.pkl', 'wb') as f:
        pickle.dump(svm_classifier, f)

    # Save the label encoding mapping
    with open('label_mapping.pkl', 'wb') as f:
        pickle.dump(label_mapping, f)
        
    print("Saved\n\nCOMPLETED")

main()

20 Images added for Harry
20 Images added for Phil
40 Images added in total 
DONE

Extracting LBM features
DONE

Training SVM classifier
DONE

Testing SVM Classifier
[1, 1, 1, 0, 0, 0, 1, 0]
Accuracy: 0.625
DONE

Saving Classifier
Saved

COMPLETED


Linear Binary patteren facial recognition

In [2]:
import pickle
import cv2
import pickle
import cv2
import numpy as np
from skimage.feature import local_binary_pattern

ROI_SIZE_XY=200
# Load the SVM classifier
with open('svm_classifier.pkl', 'rb') as f:
    svm_classifier = pickle.load(f)

# Load the label mappingqq
with open('label_mapping.pkl', 'rb') as f:
    label_mapping = pickle.load(f)
    
def getName(image):
    
    print("SHAPE",image.shape)
    if image.shape!=(200,200,3):
        print("BOX IN BAD PLACE")
        return "TOO CLOSE TO EDGE"
    # Extract LBP features
    print("Extracting features")
    LBP_features = extract_LBP_features(image)
    # Predict the label for the new image
    print("Predicting Label")
    predicted_label = svm_classifier.predict(LBP_features)[0]
    # Map the predicted label to the corresponding name using the label mapping dictionary
    print("Calculating name")
    predicted_name = label_mapping[predicted_label]

    # Print the predicted name
    print("Predicted name:", predicted_name)
    return predicted_name

def extract_LBP_features(image):
    lbp_features = []
    print("LBP FEATREUS IMAGE SHAPE",image.shape)

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    radius = 3
    n_points = 8 * radius
    lbp = local_binary_pattern(gray_image, n_points, radius, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(2**n_points + 1), density=True)
    lbp_features.append(hist)

    lbp_features = np.array(lbp_features)
    return lbp_features

def draw_name(x,y,name,frame):
    top_left=(x-10,ROI_SIZE_XY-20)
    font = cv2.FONT_HERSHEY_PLAIN
    font_scale = 3
    text_thickness = 2
    text = name
    text_org = (int(top_left[0]+ROI_SIZE_XY/4),int(top_left[1]+ROI_SIZE_XY/4))  # Place the text just above the rectangle
    color=(0,255,0)
    if name == "TOO CLOSE TO EDGE":
        text_thickness=4
        color=(0,0,255)
        text_org = (100,int(frame_height/2))  # Place the text just above the rectangle
        cv2.putText(frame, name, text_org, font, font_scale, color, text_thickness)
    else:
        cv2.putText(frame, text, text_org, font, font_scale, color, text_thickness)
    
def main():

    ##Load cascade classifier    
    cascade_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    camera = cv2.VideoCapture(1,cv2.CAP_DSHOW)
    
    while True:
        
        success, frame = camera.read()
        if success:   
            cv2.imshow("Webcam", frame) # This will open an independent window
                    
            greyscale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = cascade_classifier.detectMultiScale(greyscale, 1.3, 5)            
            for (x,y,w,h) in faces:
                cv2.imshow("Webcam", frame) # This will open an independent window
                
            
                roi = frame[y:y+ROI_SIZE_XY, x:x+ROI_SIZE_XY]
                
                name=getName(roi)
                draw_name(x,y,name,frame)
                cv2.imshow("ROI"+name, roi) # This will open an independent window
                cv2.rectangle(frame, (x-1,y-1), (x+w,y+h), (0,255,0), 3)
                cv2.imshow("Webcam", frame) # This will open an independent window
                    
                
        if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
            camera.release()
            cv2.destroyAllWindows()
            break
        
    cv2.destroyAllWindows()
main()

SHAPE (200, 200, 3)
Extracting features
LBP FEATREUS IMAGE SHAPE (200, 200, 3)
Predicting Label
Calculating name
Predicted name: Harry
SHAPE (200, 200, 3)
Extracting features
LBP FEATREUS IMAGE SHAPE (200, 200, 3)
Predicting Label
Calculating name
Predicted name: Harry
SHAPE (200, 200, 3)
Extracting features
LBP FEATREUS IMAGE SHAPE (200, 200, 3)
Predicting Label
Calculating name
Predicted name: Harry
SHAPE (200, 200, 3)
Extracting features
LBP FEATREUS IMAGE SHAPE (200, 200, 3)
Predicting Label
Calculating name
Predicted name: Harry
SHAPE (200, 200, 3)
Extracting features
LBP FEATREUS IMAGE SHAPE (200, 200, 3)
Predicting Label
Calculating name
Predicted name: Harry
